<a href="https://colab.research.google.com/github/Kruszylo/image_processing/blob/master/Copy_of_RPIntroNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up environment

1. Downloading all packages, importing them and preparing our environment
2. Remember to choose GPU runtime type

In [0]:
%matplotlib inline
!pip install --no-cache-dir -I pillow
from os import path
!pip install -q torch==1.1.0 torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [0]:
import numpy as np
import torchvision

from matplotlib import pyplot as plt;plt.style.use('default')
from torchvision.datasets import FashionMNIST

In [0]:
! pip install tensorboardX

In [0]:
! pip install scikit-image --upgrade

# restart your runtime!

In [0]:
! mkdir data
! mkdir tensorboard

Downloading the data, FashinMNiST, check: https://github.com/zalandoresearch/fashion-mnist and **make it automatically transformed to tensor**

In [0]:
from torchvision.transforms import ToTensor

In [0]:
ds = FashionMNIST('./data', train=True, target_transform=None, download=True, transform=#TODO)
ds_test = FashionMNIST('./data', train=False, target_transform=None, download=True, transform=#TODO)

Checking the data


**Write a code tha plots a few images, check the number of channels, shapes, samples in test and train dataset**

In [0]:
x, y = ds[1]
x.shape, y

In [0]:
len(ds)

In [0]:
plt.figure(figsize=(9, 9))

for i in range(9):
    #TODO

plt.show()

**Building** a NN

**Write a simple neural network consisted of 4 FullyConnected units and with chosen activation function**

In [0]:
from torch import functional
from torch import nn
from torch import optim


In [0]:
class SimpleNeuralNetwork(nn.Module):
    
    def __init__(self):
        super(SimpleNeuralNetwork, self).__init__()
        #TODO
        
    def forward(self, input_data):
        #TODO
        return x

Training a NN

**Write a code which in 10 epochs train the network. zero the gradients, calculate loss and make a backward step optimization.**

**Remember to define optimizer, criterion and to monitor loss using simple print function provided**

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader

In [0]:
batch_size = 1024

In [0]:
train_dl = DataLoader(ds, batch_size, shuffle=True)
valid_dl = DataLoader(ds_test, batch_size, shuffle=True)

In [0]:
model = SimpleNeuralNetwork()
model = model.cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam()
print(model)
number_of_epochs = 10

In [0]:
import torch

In [0]:
some_index = 0
for i in range(number_of_epochs):
    train_loss = 0
    for iteration, sample in enumerate(train_dl):
        #TODO
        
    val_loss = 0
    y_predicted = []
    y_true = []

    # disable gradinet calculations
    with torch.no_grad():
        for iteration, sample in enumerate(valid_dl):
            imgs, y = sample
            #imgs = imgs.cuda()
            #y = y.cuda()

            #TODO
            
    val_acc = accuracy_score(y_true, y_predicted)
    print('#Epoch: {}, train loss: {}, val loss: {}, val_acc: {}'.format(i, train_loss, val_loss, val_acc))
    


## Simple CNN model

**Define a conv block that contains 2 convolution operations and than MaxPooling (ot any other Pooling), convolution should preserve the image shape!**

**Finish the network with 3 Fully Connected layers**


In [0]:
class SimpleConvNeuralNetwork(nn.Module):
    
    def __init__(self):
        super(SimpleConvNeuralNetwork, self).__init__()
        #TODO
        
    def forward(self, input_data):
        
        #TODO
        return x


**Define params for training and train the model for 10 epochs**

In [0]:
batch_size = 1024
train_dl = DataLoader(#TODO)
valid_dl = DataLoader(#TODO)
model = SimpleConvNeuralNetwork()
model = model.cuda()

criterion = #TODO
optimizer = #TODO
print(model)
number_of_epochs = 10

In [0]:
some_index = 0
for i in range(number_of_epochs):
    train_loss = 0
    for iter, sample in enumerate(train_dl):
        #TODO
        
    val_loss = 0
    y_predicted = []
    y_true = []

    # disable gradinet calculations
    with torch.no_grad():
        for iteration, sample in enumerate(valid_dl):
            #TODO
            
    val_acc = accuracy_score(y_true, y_predicted)
    print('#Epoch: {}, train loss: {}, val loss: {}, val_acc: {}'.format(i, train_loss, val_loss, val_acc))
    

**Write LeNet-5 classifier and train it https://engmrk.com/lenet-5-a-classic-cnn-architecture/**

In [0]:
#TODO